In [69]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
# from scipy.stats import norm, multivariate_normal
# python 中的优化算法集合
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
from scipy.optimize import fmin_bfgs
import matplotlib.pyplot as plt
plt.style.use("seaborn-notebook")
%matplotlib inline
# import matplotlib as mpl
# mpl.rcParams["font.family"] = "DejaVu Sans"
import seaborn as sns
sns.set(font="DejaVu Sans")

In [2]:
params = sio.loadmat("data/ex8_movieParams.mat")
params.keys()

dict_keys(['__header__', '__globals__', 'num_movies', 'num_features', 'num_users', 'Theta', 'X', '__version__'])

In [3]:
movie = sio.loadmat("data/ex8_movies.mat")
movie.keys()

dict_keys(['__header__', '__globals__', 'R', 'Y', '__version__'])

In [24]:
# user gave movie rate or not
R = pd.DataFrame(movie.get("R"))
print(R.shape)
R.head()

(1682, 943)


,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1,1,0,0,1,1,0,0,0,1,...,1,1,1,0,1,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
for i in range(len(R.columns)):
    cc = sum(R.ix[R.ix[:, i].isnull(), i])
    if cc > 0:
        print(i, cc)

In [25]:
# rate matrix
Y = pd.DataFrame(movie.get("Y"))
print(Y.shape)
Y.head()

(1682, 943)


,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,5,4,0,0,4,4,0,0,0,4,...,2,3,4,0,4,0,0,5,0,0
1,3,0,0,0,3,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,5
2,4,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
3,3,0,0,0,0,0,5,0,0,4,...,5,0,0,0,0,0,2,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
for i in range(len(Y.columns)):
    cc = sum(Y.ix[Y.ix[:, i].isnull(), i])
    if cc > 0:
        print(i, cc)

In [12]:
def load_mat(mat_file, vals):
    
    dat = sio.loadmat(mat_file)
    return (pd.DataFrame(dat.get(v)) for v in vals)

In [14]:
# params.keys()
num_movies, num_features, num_users, Theta, X = load_mat("data/ex8_movieParams.mat", ["num_movies", "num_features", "num_users", "Theta", "X"])

In [16]:
print(num_movies)
print(num_features)
print(num_users)

      0
0  1682
    0
0  10
     0
0  943


In [18]:
print(Theta.shape)
Theta.head()

(943, 10)


,0,1,2,3,4,5,6,7,8,9
0,0.285444,-1.684265,0.262939,-0.287317,0.585725,0.980188,-0.063375,0.767232,-1.104602,-0.251867
1,0.505013,-0.454648,0.317462,-0.115087,0.567704,0.818905,0.461649,1.093063,-1.200294,-0.391617
2,-0.431917,-0.478804,0.846711,-0.011731,-0.138578,0.742591,-0.440276,1.363340,-0.325235,-0.194682
3,0.728598,-0.271894,0.326844,-0.250724,-0.471017,1.215827,-0.106129,1.615801,-0.670805,0.129868
4,0.052119,-1.380697,0.637941,-0.231980,0.246337,1.438879,-0.860022,0.668077,-0.560330,0.414959


In [19]:
print(X.shape)
X.head()

(1682, 10)


,0,1,2,3,4,5,6,7,8,9
0,1.048686,-0.400232,1.194119,0.371128,0.407607,0.974407,-0.058410,0.861721,-0.697290,0.288746
1,0.780851,-0.385626,0.521198,0.227355,0.570109,0.641264,-0.550006,0.704021,-0.485835,-0.564624
2,0.641509,-0.547854,-0.083796,-0.598519,-0.017694,0.299736,-0.720807,0.838546,-0.694832,-1.134796
3,0.453618,-0.800218,0.680481,-0.081743,0.136601,0.907561,0.277682,0.369300,-1.261208,-0.235581
4,0.937538,0.106090,0.361953,0.086646,0.287505,0.518644,-0.056871,0.914573,-0.819334,-0.542847


In [21]:
sum(X.isnull())

45

In [63]:
np.dot(Theta.ix[0, :].transpose(), X.ix[0, :])

3.7369778735669072

In [71]:
sum(Y * R)

444153

In [72]:
sum(Y)

444153

In [78]:
a = np.reshape(np.array([1, 2, 3, 4]), (2, 2))
b = np.reshape(np.array([0, 1]), (2, 1))

[[1 2]
 [3 4]]
[[0]
 [1]]


In [98]:
np.reshape(a, (4, 1))[:, 0]

array([1, 2, 3, 4])

In [99]:
np.reshape(b, (2, 1))[:, 0]

array([0, 1])

In [108]:
a.flatten()

array([1, 2, 3, 4])

In [102]:
c = np.hstack((np.reshape(a, (4, 1))[:, 0], np.reshape(b, (2, 1))[:, 0]))
c

array([1, 2, 3, 4, 0, 1])

In [109]:
c = np.append(a.flatten(), b.flatten())
c

array([1, 2, 3, 4, 0, 1])

In [103]:
np.reshape(c[:4], (2, 2))

array([[1, 2],
       [3, 4]])

In [105]:
np.reshape(c[4:], (2, 1))

array([[0],
       [1]])

In [107]:
np.zeros((2, 2))

array([[ 0.,  0.],
       [ 0.,  0.]])

In [132]:
print(X.shape)
print(Theta.shape)
# np.dot(X, Theta.T)   # 1682 * 10 x 10 * 943
# print(Y.shape - np.dot(X, Theta.T))
np.sum(((np.asarray(Y) - np.dot(X, Theta.T)) * np.asarray(R))**2)

(1682, 10)
(943, 10)


55837.280249088421

In [135]:
np.where(np.asarray(R)[1, :] == 1)[0]

array([  0,   4,  12,  21,  29,  41,  48,  63,  71,  82,  86,  91,  94,
       101, 109, 129, 177, 192, 196, 199, 200, 206, 212, 216, 221, 233,
       248, 249, 255, 266, 267, 270, 275, 278, 279, 291, 292, 300, 302,
       304, 319, 324, 326, 345, 362, 372, 373, 377, 378, 384, 386, 392,
       397, 398, 404, 406, 415, 424, 428, 434, 441, 449, 454, 465, 471,
       483, 486, 494, 496, 505, 520, 531, 535, 542, 550, 560, 565, 599,
       617, 620, 621, 626, 631, 639, 641, 642, 647, 649, 652, 654, 659,
       670, 681, 685, 704, 708, 714, 726, 737, 745, 748, 750, 756, 763,
       772, 773, 789, 794, 795, 797, 803, 805, 806, 814, 825, 829, 843,
       845, 863, 867, 869, 879, 885, 888, 891, 895, 898, 915, 923, 933, 942])

In [68]:
def cofiCostFunc(params, Y, R, num_movies, num_users, num_features, lambda_p):
    
    """
    # X:       movies * features
    # Theta:   users * features
    params:    a vector contains X and Theta
    Y:         movies * users
    R:         movies * users if one user rate corresponding movie
    lambda_p:  lambda for regularization
    J:         returned cost
    """
    # extract vectors
    X = np.reshape(params[:(num_movies * num_features + 1)], (num_movies, num_features))
    Theta = np.reshape(params[(num_movies * num_features + 1):], (num_users, num_features))
    
    # return vals
    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)
    
    # compute the cost
    J = 0.5 * np.sum(
        ( (np.asarray(Y) - np.dot(X, Theta.T)) * np.asarray(R))**2
    ) +    # regularization of X
    0.5 * lambda_p * np.sum(Theta**2) +
    0.5 * lambda_p * np.sum(X**2)
    
    # compute the X gradient
    for i in range(num_movies):
        # users who had rate for a moive
        idx = np.where(np.asarray(R)[i, :] == 1)[0]
        # those users theta paramter for movies
        temp_theta = Theta[idx, :]
        # rating given to the movie
        temp_y = Y[, idx]
    
    return J

In [65]:
# movie, user, feature number
mn, un, fn = (4, 5, 3)
sX = X.ix[:mn, :fn]
sTheta = Theta.ix[:un, :fn]
sY = Y.ix[:mn, :un]
sR = R.ix[:mn, :un]

In [ ]:
fmin_bfgs(cost_without_regularization, )